<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Structure-cheat-sheet" data-toc-modified-id="Structure-cheat-sheet-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Structure cheat sheet</a></span></li><li><span><a href="#Data-structure" data-toc-modified-id="Data-structure-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Data structure</a></span></li><li><span><a href="#get-features" data-toc-modified-id="get-features-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>get features</a></span></li><li><span><a href="#Exploration-of-non-spectral-Features" data-toc-modified-id="Exploration-of-non-spectral-Features-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Exploration of non-spectral Features</a></span></li><li><span><a href="#Exploring-spectral-features" data-toc-modified-id="Exploring-spectral-features-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Exploring spectral features</a></span></li></ul></div>

# Basics

## Structure cheat sheet

1. func: train data lead (following order)
    1. read the descriptive dataframe from the feature-pipeline
    2. extract feature from the feature-objects which are labeled train-dataset from dataframe
    3. create numpy feature array for the processing pipeline
2. preprocessing
    1. Transformation (any combination of the following)
        + log-transform
        + PCA
        + others
    2. Scaling (one of the following)
        + StandardScaler
        + MinMaxScaler
3. Unsupervised Clustering
    1. Estimate initial hyperparameter
    2. Create grid over various hyperparameters
    3. Train all and choose the best according to metric
    
    
in all steps the cluster-recorder object (possibly dataframe-row) will record all the meta-information like hyper-parameters

## Data structure

There are multiple degrees of freedom in the data:

1. Signal to noise ratio (SNR)
2. Machine type
    1. pump
    2. fan
    3. valve (solenoid)
    4. slider
3. Machine ID
    1. four different machine IDs
    
The pipeline will be applied to fixed SNR, fixed machine type and fixed ID

## get features

Get the descriptive dataframe for the features.

The descriptive dataframe contains all IDs of the pump. We will focus on ID '00' for now since the modeling phase is seperated per SNR, per machine, per ID anyway.

class: 
+ uni\_\<model\>
attributes:
+ default threshold
+ roc_auc
methods:
+ fit
+ predict
+ predict_score
+ eval_roc_auc

In [1]:
#===============================================
# Basic Imports
BASE_FOLDER = '../../'
%run -i ..\..\utility\feature_extractor\JupyterLoad_feature_extractor.py
%run -i ..\..\utility\modeling\JupyterLoad_modeling.py

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA, FastICA
from tqdm.auto import tqdm
import glob

load feature_extractor_mother
load feature_extractor_mel_spectra
load feature_extractor_psd
load feature_extractor_ICA2
load feature_extractore_pre_nnFilterDenoise
load extractor_diagram_mother
load Simple_FIR_HP
load TimeSliceAppendActivation
load load_data
Load split_data
Load anomaly_detection_models
Load pseudo_supervised_models
Load tensorflow models
Load detection_pipe


The exploration of the Dimensionality Reduction was already done in another notebook. We derived the following rules of thumb:

1. PCA and ICA deliver almost the same results looking at the relative absolute error
2. PCA is usually much faster
3. on PSD and the ICA demix matrix, no dimensionality reduction is needed
4. for a framed Mel-spectrum a number of components between 32 and 64 is a good measure. The resulting error is about 2-4%
5. for a whole Mel-spectrum a number of components between 64 and 128 is advised

## Exploration of non-spectral Features

## Exploring spectral features

In [17]:
diagram = 'extdia_v1'
machines = ['pump', 'fan', 'slider'
            #, 'valve'
]
SNRs = ['6dB', 'min6dB']
IDs = ['00', '02'
    #, '04', '06'
      ]

features = [('MEL_den', {'function':'frame', 'frames':5})
            #, ('PSD_raw', {'function':'channel'})
            ]

tasks = [{
        'path_descr':glob.glob(BASE_FOLDER 
                            + '/dataset/extdia_v1*/{}{}{}_EDiaV1'.format(machine, SNR, ID) 
                            + "*pandaDisc*.pkl", recursive=True)[-1],
        'feat':feature[1], 
        'feat_col':feature[0], 
        'SNR':SNR, 
        'machine':machine, 
        'ID':ID,
        'BASE_FOLDER':BASE_FOLDER}
        for machine in machines
        for SNR in SNRs
        for ID in IDs
        for feature in features
        ]

preprocessing = [
    (PCA, {'n_components':64}),
    (StandardScaler, {})
]

modeling = (uni_AutoEncoder, {'epochs':50})

pipes = [Pipe(preprocessing, modeling) for i in range(len(features)*len(IDs)*len(machines)*len(SNRs))]

features = [('MEL_den', {'function':'frame', 'frames':5})]

tasks += [{
        'path_descr':glob.glob(BASE_FOLDER 
                            + '/dataset/extdia_v1*/{}{}{}_EDiaV1'.format(machine, SNR, ID) 
                            + "*pandaDisc*.pkl", recursive=True)[-1],
        'feat':feature[1], 
        'feat_col':feature[0], 
        'SNR':SNR, 
        'machine':machine, 
        'ID':ID,
        'BASE_FOLDER':BASE_FOLDER}
        for machine in machines
        for SNR in SNRs
        for ID in IDs
        for feature in features
        ]

preprocessing = [
    (PCA, {'n_components':64}),
    (StandardScaler, {})
]

modeling = (uni_IsolationForest, {'n_estimators':64, 'max_features':4})

pipes += [Pipe(preprocessing, modeling) for i in range(len(features)*len(IDs)*len(machines)*len(SNRs))]

features = [('PSD_raw', {'function':'channel'})]

tasks += [{
        'path_descr':glob.glob(BASE_FOLDER 
                            + '/dataset/extdia_v1*/{}{}{}_EDiaV1'.format(machine, SNR, ID) 
                            + "*pandaDisc*.pkl", recursive=True)[-1],
        'feat':feature[1], 
        'feat_col':feature[0], 
        'SNR':SNR, 
        'machine':machine, 
        'ID':ID,
        'BASE_FOLDER':BASE_FOLDER}
        for machine in machines
        for SNR in SNRs
        for ID in IDs
        for feature in features
        ]

preprocessing = [
    (PCA, {'n_components':64}),
    (StandardScaler, {})
]

modeling = (uni_IsolationForest, {'n_estimators':200, 'max_features':1})

pipes += [Pipe(preprocessing, modeling) for i in range(len(features)*len(IDs)*len(machines)*len(SNRs))]

12

In [21]:
#failed = 3, 7
start = 0
for pipe, task in tqdm(zip(pipes[start:], tasks[start:]), total=len(tasks[start:])):
    pipe.run_pipe(task)

../..//dataset\extdia_v1\pump6dB00_EDiaV1HPaug0_pandaDisc.pkl --> Done
...loading data
data loading completed

...preprocessing data
data preprocessing finished

...fitting the model
Epoch 1/50
1042/1042 [==============================] - 10s 10ms/step - loss: 0.9435
Epoch 2/50
1042/1042 [==============================] - 7s 6ms/step - loss: 0.9003
Epoch 3/50
1042/1042 [==============================] - 7s 6ms/step - loss: 0.8778
Epoch 4/50
1042/1042 [==============================] - 7s 7ms/step - loss: 0.8560
Epoch 5/50
1042/1042 [==============================] - 7s 6ms/step - loss: 0.8386
Epoch 6/50
1042/1042 [==============================] - 7s 6ms/step - loss: 0.8229
Epoch 7/50
1042/1042 [==============================] - 7s 7ms/step - loss: 0.8062
Epoch 8/50
1042/1042 [==============================] - 7s 7ms/step - loss: 0.7958
Epoch 9/50
1042/1042 [==============================] - 7s 7ms/step - loss: 0.7873
Epoch 10/50
1042/1042 [==============================] - 7s 6ms/step

540/540 [==============================] - 5s 9ms/step - loss: 0.6730
Epoch 44/50
540/540 [==============================] - 4s 8ms/step - loss: 0.6714
Epoch 45/50
540/540 [==============================] - 5s 9ms/step - loss: 0.6700
Epoch 46/50
540/540 [==============================] - 5s 9ms/step - loss: 0.6684
Epoch 47/50
540/540 [==============================] - 5s 9ms/step - loss: 0.6672
Epoch 48/50
540/540 [==============================] - 5s 9ms/step - loss: 0.6661
Epoch 49/50
540/540 [==============================] - 5s 8ms/step - loss: 0.6648
Epoch 50/50
540/540 [==============================] - 4s 8ms/step - loss: 0.6635
model fitted successfully

...evaluating model
evaluation successfull, roc_auc: 0.9944236291566749
pipe saved to pickle
../..//dataset\extdia_v1\pumpmin6dB00_EDiaV1HPaug0_pandaDisc.pkl --> Done
...loading data
data loading completed

...preprocessing data
data preprocessing finished

...fitting the model
Epoch 1/50
1042/1042 [============================

540/540 [==============================] - 6s 11ms/step - loss: 0.6662
Epoch 35/50
540/540 [==============================] - 6s 11ms/step - loss: 0.6646
Epoch 36/50
540/540 [==============================] - 5s 10ms/step - loss: 0.6630
Epoch 37/50
540/540 [==============================] - 5s 10ms/step - loss: 0.6609
Epoch 38/50
540/540 [==============================] - 6s 10ms/step - loss: 0.6598
Epoch 39/50
540/540 [==============================] - 6s 10ms/step - loss: 0.6583
Epoch 40/50
540/540 [==============================] - 6s 11ms/step - loss: 0.6575
Epoch 41/50
540/540 [==============================] - 6s 10ms/step - loss: 0.6562
Epoch 42/50
540/540 [==============================] - 5s 10ms/step - loss: 0.6555
Epoch 43/50
540/540 [==============================] - 6s 11ms/step - loss: 0.6550
Epoch 44/50
540/540 [==============================] - 5s 10ms/step - loss: 0.6537
Epoch 45/50
540/540 [==============================] - 6s 11ms/step - loss: 0.6529
Epoch 46/50
540/

397/397 [==============================] - 3s 8ms/step - loss: 0.7725
Epoch 27/50
397/397 [==============================] - 3s 8ms/step - loss: 0.7718
Epoch 28/50
397/397 [==============================] - 3s 8ms/step - loss: 0.7710
Epoch 29/50
397/397 [==============================] - 3s 8ms/step - loss: 0.7699
Epoch 30/50
397/397 [==============================] - 3s 8ms/step - loss: 0.7689
Epoch 31/50
397/397 [==============================] - 3s 7ms/step - loss: 0.7684
Epoch 32/50
397/397 [==============================] - 3s 8ms/step - loss: 0.7674
Epoch 33/50
397/397 [==============================] - 3s 7ms/step - loss: 0.7665
Epoch 34/50
397/397 [==============================] - 3s 8ms/step - loss: 0.7639
Epoch 35/50
397/397 [==============================] - 3s 8ms/step - loss: 0.7607
Epoch 36/50
397/397 [==============================] - 3s 8ms/step - loss: 0.7587
Epoch 37/50
397/397 [==============================] - 3s 8ms/step - loss: 0.7570
Epoch 38/50
397/397 [=======

397/397 [==============================] - 4s 9ms/step - loss: 0.6823
Epoch 19/50
397/397 [==============================] - 3s 8ms/step - loss: 0.6794
Epoch 20/50
397/397 [==============================] - 3s 8ms/step - loss: 0.6766
Epoch 21/50
397/397 [==============================] - 3s 8ms/step - loss: 0.6744
Epoch 22/50
397/397 [==============================] - 3s 8ms/step - loss: 0.6724
Epoch 23/50
397/397 [==============================] - 3s 8ms/step - loss: 0.6708
Epoch 24/50
397/397 [==============================] - 4s 9ms/step - loss: 0.6687
Epoch 25/50
397/397 [==============================] - ETA: 0s - loss: 0.667 - 3s 9ms/step - loss: 0.6672
Epoch 26/50
397/397 [==============================] - 3s 8ms/step - loss: 0.6656
Epoch 27/50
397/397 [==============================] - 3s 8ms/step - loss: 0.6640
Epoch 28/50
397/397 [==============================] - 3s 8ms/step - loss: 0.6627
Epoch 29/50
397/397 [==============================] - 3s 9ms/step - loss: 0.6617
Epoc

262/262 [==============================] - 2s 8ms/step - loss: 0.6704
Epoch 11/50
262/262 [==============================] - 2s 9ms/step - loss: 0.6593
Epoch 12/50
262/262 [==============================] - 2s 8ms/step - loss: 0.6508
Epoch 13/50
262/262 [==============================] - 2s 8ms/step - loss: 0.6428
Epoch 14/50
262/262 [==============================] - 2s 8ms/step - loss: 0.6362
Epoch 15/50
262/262 [==============================] - 2s 8ms/step - loss: 0.6293
Epoch 16/50
262/262 [==============================] - 2s 8ms/step - loss: 0.6234
Epoch 17/50
262/262 [==============================] - 2s 8ms/step - loss: 0.6181
Epoch 18/50
262/262 [==============================] - 2s 8ms/step - loss: 0.6140
Epoch 19/50
262/262 [==============================] - 2s 8ms/step - loss: 0.6098
Epoch 20/50
262/262 [==============================] - 2s 8ms/step - loss: 0.6065
Epoch 21/50
262/262 [==============================] - 2s 8ms/step - loss: 0.6026
Epoch 22/50
262/262 [=======

262/262 [==============================] - 2s 7ms/step - loss: 0.8989
Epoch 3/50
262/262 [==============================] - 2s 7ms/step - loss: 0.8553
Epoch 4/50
262/262 [==============================] - 2s 7ms/step - loss: 0.8202
Epoch 5/50
262/262 [==============================] - 2s 7ms/step - loss: 0.7893
Epoch 6/50
262/262 [==============================] - 2s 7ms/step - loss: 0.7643
Epoch 7/50
262/262 [==============================] - 2s 7ms/step - loss: 0.7431
Epoch 8/50
262/262 [==============================] - 2s 7ms/step - loss: 0.7268
Epoch 9/50
262/262 [==============================] - 2s 7ms/step - loss: 0.7126
Epoch 10/50
262/262 [==============================] - 2s 7ms/step - loss: 0.7017
Epoch 11/50
262/262 [==============================] - 2s 6ms/step - loss: 0.6913
Epoch 12/50
262/262 [==============================] - 2s 6ms/step - loss: 0.6821
Epoch 13/50
262/262 [==============================] - 2s 7ms/step - loss: 0.6733
Epoch 14/50
262/262 [==============

model fitted successfully

...evaluating model
evaluation successfull, roc_auc: 0.9873386900413927
pipe saved to pickle
../..//dataset\extdia_v1\pumpmin6dB00_EDiaV1HPaug0_pandaDisc.pkl --> Done
...loading data
data loading completed

...preprocessing data
data preprocessing finished

...fitting the model
model fitted successfully

...evaluating model
evaluation successfull, roc_auc: 0.7912367352926793
pipe saved to pickle
../..//dataset\extdia_v1\pumpmin6dB02_EDiaV1HP_pandaDisc.pkl --> Done
...loading data
data loading completed

...preprocessing data
data preprocessing finished

...fitting the model
model fitted successfully

...evaluating model
evaluation successfull, roc_auc: 0.8479831182533885
pipe saved to pickle
../..//dataset\extdia_v1\fan6dB00_EDiaV1HP_pandaDisc.pkl --> Done
...loading data
data loading completed

...preprocessing data
data preprocessing finished

...fitting the model
model fitted successfully

...evaluating model
evaluation successfull, roc_auc: 0.879377478886

In [30]:
glob.glob(BASE_FOLDER + '/dataset/extdia_v1*/{}{}{}_EDiaV1'.format('valve', 'min6dB', '06') 
                            + "*pandaDisc*.pkl", recursive=True)

['../..//dataset\\extdia_v1_sporadic\\valvemin6dB06_EDiaV1HPTsSl_pandaDisc.pkl',
 '../..//dataset\\extdia_v1_sporadic\\valvemin6dB06_EDiaV1HP_pandaDisc.pkl']

In [31]:
diagram = 'extdia_v1'
machines = [#'pump', 'fan', 'slider',
            'valve'
]
SNRs = [#'6dB', 
    'min6dB']
IDs = [#'00', '02', '04', 
    '06'
      ]

features = [('MEL_den', {'function':'frame', 'frames':5})
            #, ('PSD_raw', {'function':'channel'})
            ]

tasks = [{
        'path_descr':glob.glob(BASE_FOLDER 
                            + '/dataset/extdia_v1*/{}{}{}_EDiaV1'.format(machine, SNR, ID) 
                            + "*pandaDisc*.pkl", recursive=True)[-1],
        'feat':feature[1], 
        'feat_col':feature[0], 
        'SNR':SNR, 
        'machine':machine, 
        'ID':ID,
        'BASE_FOLDER':BASE_FOLDER}
        for machine in machines
        for SNR in SNRs
        for ID in IDs
        for feature in features
        ]

preprocessing = [
    (PCA, {'n_components':64}),
    (StandardScaler, {})
]

modeling = (uni_AutoEncoder, {'epochs':50})

pipes = [Pipe(preprocessing, modeling) for i in range(len(features)*len(IDs)*len(machines)*len(SNRs))]

features = [('MEL_den', {'function':'frame', 'frames':5})]

tasks += [{
        'path_descr':glob.glob(BASE_FOLDER 
                            + '/dataset/extdia_v1*/{}{}{}_EDiaV1'.format(machine, SNR, ID) 
                            + "*pandaDisc*.pkl", recursive=True)[0],
        'feat':feature[1], 
        'feat_col':feature[0], 
        'SNR':SNR, 
        'machine':machine, 
        'ID':ID,
        'BASE_FOLDER':BASE_FOLDER}
        for machine in machines
        for SNR in SNRs
        for ID in IDs
        for feature in features
        ]

preprocessing = [
    (PCA, {'n_components':64}),
    (StandardScaler, {})
]

modeling = (uni_IsolationForest, {'n_estimators':64, 'max_features':4})

pipes += [Pipe(preprocessing, modeling) for i in range(len(features)*len(IDs)*len(machines)*len(SNRs))]

features = [('PSD_raw', {'function':'channel'})]

tasks += [{
        'path_descr':glob.glob(BASE_FOLDER 
                            + '/dataset/extdia_v1*/{}{}{}_EDiaV1'.format(machine, SNR, ID) 
                            + "*pandaDisc*.pkl", recursive=True)[0],
        'feat':feature[1], 
        'feat_col':feature[0], 
        'SNR':SNR, 
        'machine':machine, 
        'ID':ID,
        'BASE_FOLDER':BASE_FOLDER}
        for machine in machines
        for SNR in SNRs
        for ID in IDs
        for feature in features
        ]

preprocessing = [
    (PCA, {'n_components':64}),
    (StandardScaler, {})
]

modeling = (uni_IsolationForest, {'n_estimators':200, 'max_features':1})

pipes += [Pipe(preprocessing, modeling) for i in range(len(features)*len(IDs)*len(machines)*len(SNRs))]

In [32]:
start = 0
for pipe, task in tqdm(zip(pipes[start:3], tasks[start:]), total=len(tasks[start:3])):
    pipe.run_pipe(task)

../..//dataset\extdia_v1_sporadic\valvemin6dB06_EDiaV1HP_pandaDisc.pkl --> Done
...loading data


ValueError: could not broadcast input array from shape (76,400) into shape (309,400)

In [26]:
len(pipes)

3